In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

dataset_folder = '/kaggle/input/smai-25-sec-a-project-phase-2-lat-long-prediction/'

# Define paths - update these to your actual paths
model_path = '/kaggle/input/best_convnext_model/pytorch/default/1/best_convnext_model.pth'  # Path to your saved model
train_csv_path = dataset_folder+'labels_train.csv'  # Original training CSV
val_csv_path = dataset_folder+'labels_val.csv'  # Original validation CSV
train_img_dir = dataset_folder+'images_train/images_train'  # Training images directory
val_img_dir = dataset_folder+'images_val/images_val'  # Validation images directory
output_train_csv = 'predicted_regions_train.csv'  # Output predicted regions for training set
output_val_csv = 'predicted_regions_val.csv'  # Output predicted regions for validation set

# Load original data to maintain structure
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)

# Image transformation for prediction (same as validation transform)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset class for prediction
class PredictionDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data_frame = csv_file
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 0]  # Assuming filename is in first column
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            print(f"Error loading image: {img_path}")
            # Return a placeholder image in case of error
            image = Image.new('RGB', (256, 256), color='gray')
            
        if self.transform:
            image = self.transform(image)
            
        # Return image and index (to map back to dataframe)
        return image, idx

# Create prediction datasets
train_pred_dataset = PredictionDataset(train_df, train_img_dir, transform)
val_pred_dataset = PredictionDataset(val_df, val_img_dir, transform)

# DataLoaders for prediction
batch_size = 32  # Can use larger batch size for prediction
train_pred_loader = DataLoader(train_pred_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
val_pred_loader = DataLoader(val_pred_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the model architecture (must match how it was defined during training)
model = models.convnext_base(weights=None)  # No need to download weights
num_ftrs = model.classifier[2].in_features
model.classifier = nn.Sequential(
    model.classifier[0],
    model.classifier[1],
    nn.Dropout(0.2),
    nn.Linear(num_ftrs, 15)  # 15 regions (0-14 for PyTorch)
)

# Load the saved model weights
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

def predict_regions(dataloader, dataframe):
    """Make region predictions for all images in dataloader"""
    predictions = []
    indices = []
    
    with torch.no_grad():
        for images, idx in tqdm(dataloader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            # Get predicted class (region)
            _, preds = torch.max(outputs, 1)
            
            predictions.extend(preds.cpu().numpy())
            indices.extend(idx.numpy())
    
    # Convert from 0-14 back to 1-15 for Region_ID
    predictions = [p + 1 for p in predictions]
    
    # Create a DataFrame with predictions
    result_df = dataframe.copy()
    result_df.loc[indices, 'Region_ID'] = predictions
    
    return result_df

# Predict regions for training set
print("Predicting regions for training set...")
train_pred_df = predict_regions(train_pred_loader, train_df)

# Predict regions for validation set
print("Predicting regions for validation set...")
val_pred_df = predict_regions(val_pred_loader, val_df)

# Save the predictions to CSV files
train_pred_df.to_csv(output_train_csv, index=False)
val_pred_df.to_csv(output_val_csv, index=False)

print(f"Saved predicted regions for training set to {output_train_csv}")
print(f"Saved predicted regions for validation set to {output_val_csv}")

# Print region distribution in predictions for comparison
print("\nPredicted Region ID distribution (Training):")
print(train_pred_df['Region_ID'].value_counts().sort_index())

print("\nPredicted Region ID distribution (Validation):")
print(val_pred_df['Region_ID'].value_counts().sort_index())

Using device: cuda


/tmp/ipykernel_18/2522854220.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Predicting regions for training set...


Predicting: 100%|██████████| 205/205 [01:12<00:00,  2.84it/s]


Predicting regions for validation set...


Predicting: 100%|██████████| 12/12 [00:04<00:00,  2.59it/s]

Saved predicted regions for training set to predicted_regions_train.csv
Saved predicted regions for validation set to predicted_regions_val.csv

Predicted Region ID distribution (Training):
Region_ID
1     361
2     451
3     444
4     461
5     484
6     452
7     392
8     502
9     262
10    546
11    403
12    447
13    397
14    166
15    774
Name: count, dtype: int64

Predicted Region ID distribution (Validation):
Region_ID
1     21
2     21
3     28
4     26
5     28
6     28
7     22
8     28
9     14
10    34
11    21
12    30
13    24
14     9
15    35
Name: count, dtype: int64
